## Break repeating-key XOR
### It is officially on, now.
This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.

There's a file here. It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40. <br>
2. Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between: <br>
    ```
    this is a test
    ```
    and <br>
    ```
    wokka wokka!!!
    ```
    is 37. Make sure your code agrees before you proceed. <br>
3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE. <br>
4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances. <br>
5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length. <br>
6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on. <br>
7. Solve each block as if it was single-character XOR. You already have code to do this. <br>
8. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key. <br>

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important. <br>

### No, that's not a mistake.
We get more tech support questions for this challenge than any of the other ones. We promise, there aren't any blatant errors in this text. In particular: the "wokka wokka!!!" edit distance really is 37.

In [12]:
import base64
import math

### just test

In [1]:
def hamming_d(b1, b2):
    score = 0
    for i in range(len(b1)):
        if b1[i] != b2[i]:
            score += 1
    return score

t1 = "this is a test"
t2 = "wokka wokka!!!"

t1_b = [format(ord(char), "08b") for char in t1]
t2_b = [format(ord(char), "08b") for char in t2]

result = 0

for i in range(len(t1_b)):
    score_v = hamming_d(t1_b[i], t2_b[i])
    result += score_v

print(result)

37


In [3]:
def hamming_distance(b1: bytes, b2: bytes) -> int:
    distance: int = 0
    for binary1,binary2 in zip(b1, b2):
        distance += bin(binary1 ^ binary2).count('1')
    return distance

def get_english_score_text(c_plain: bytes) -> int:
    score = 0
    for byte in c_plain:
        if byte == 32 or (97 <= byte <= 122) or (65 <= byte <= 90):
            score += 1
    return score

def single_byte_xor(block: bytes) -> int:
    best_score = 0
    best_key = 0

    for k in range(256):
        xord = [b ^ k for b in block] 
        
        score_tmp = get_english_score_text(xord)
        if score_tmp > best_score:
            best_score = score_tmp
            best_key = k
    return best_key

def repeating_key_xor(data, key):
    output = bytearray()
    for i in range(len(data)):
        output.append(data[i] ^ key[i % len(key)])
    return bytes(output) 

def break_repeating_key(chipertex: str):
    normalized_distance = []

    for key_size in range(2,40):
        chunks = [chipertex[i:i+key_size] for i in range(0,len(chipertex),key_size)]

        avg_dist = 0
        pairs = 0

        for i in range(len(chunks)):
            for j in range(i + 1, len(chunks)):
                dist = hamming_distance(chunks[i], chunks[j])
                avg_dist += dist / key_size
                pairs += 1
        if pairs > 0:
            normalized = avg_dist / pairs
            normalized_distance.append((key_size, normalized))
    
    normalized_distance.sort(key=lambda x:x[1])
    best_keysize = normalized_distance[0][0]

    print("best key: ", best_keysize)

    found_key = bytearray()
    for i in range(best_keysize):
        block = chipertex[i::best_keysize]

        key_byte = single_byte_xor(block)
        found_key.append(key_byte)
    print("Recovered key:",found_key.decode('utf-8', errors='ignore'))

    plain_text = repeating_key_xor(chipertex, found_key)
    print(plain_text.decode('utf-8', errors='ignore'))

if __name__ == "__main__":
    f = open('6.txt')
    chiper = base64.b64decode(f.read())
    break_repeating_key(chiper)
    # f.close()

NameError: name 'base64' is not defined